In [162]:
import pandas as pd
import backtesting
# from backtesting import Backtest, Strategy
from backtesting.lib import SignalStrategy, TrailingStrategy
from backtesting import *
import numpy as np
from datetime import timedelta
import warnings
warnings.filterwarnings("ignore")

import ipywidgets as widgets
import matplotlib.pyplot as plt

In [163]:
data = pd.read_csv('data/decisions_EURUSD_1H.csv')
#data = pd.read_csv('data/decisions_EURUSD_5Y.csv')


In [164]:

data = pd.read_csv('data/decisions_EURUSD_1H.csv')
#data = pd.read_csv('data/decisions_EURUSD_5Y.csv')


data = data.set_index(pd.DatetimeIndex(data['data_datetime']))
#data = data.set_index(pd.DatetimeIndex(data['Date']))

In [156]:
len(data)

1284

In [135]:
# Zliczenie przypadków, w których wartości są przeciwne

rsi_count = ((data['dec_rsi'] == 1) & (data['Signal_RSI_fuzz_system'] == -1) | (data['dec_rsi'] == -1) & (data['Signal_RSI_fuzz_system'] == 1)).sum()
print("rsi_count", rsi_count)

williams_count = ((data['dec_will'] == 1) & (data['Signal_william_fuzz_system'] == -1) | (data['dec_will'] == -1) & (data['Signal_william_fuzz_system'] == 1)).sum()
print("williams_count", williams_count)

# cross_MA - ostre decyzje
ma_count = ((data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == -1) | (data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == 1)).sum()
print("ma_count", ma_count)

macd_count = ((data['MACD_decision'] == 1) & (data['fuzz_MACD_strategy'] == -1) | (data['MACD_decision'] == -1) & (data['fuzz_MACD_strategy'] == 1)).sum()
print("macd_count", macd_count)


bb_count = ((data['dec_bollinger'] == 1) & (data['fuzz_BB_strategy'] == -1) | (data['dec_bollinger'] == -1) & (data['fuzz_BB_strategy'] == 1)).sum()

print("bb_count", bb_count)


rsi_count 0
williams_count 0
ma_count 3
macd_count 0
bb_count 0


In [116]:
# Zliczenie przypadków, w których wartości są takie same

rsi_count = ((data['dec_rsi'] == - 1) & (data['Signal_RSI_fuzz_system'] == -1) | (data['dec_rsi'] == 1) & (data['Signal_RSI_fuzz_system'] == 1)).sum()
print("rsi_count", rsi_count)

williams_count = ((data['dec_will'] == -1) & (data['Signal_william_fuzz_system'] == -1) | (data['dec_will'] == 1) & (data['Signal_william_fuzz_system'] == 1)).sum()
print("williams_count", williams_count)

# cross_MA - ostre decyzje
ma_count = ((data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == -1) | (data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == 1)).sum()
print("ma_count", ma_count)

macd_count = ((data['MACD_decision'] == -1) & (data['fuzz_MACD_strategy'] == -1) | (data['MACD_decision'] == 1) & (data['fuzz_MACD_strategy'] == 1)).sum()
print("macd_count", macd_count)


bb_count = ((data['dec_bollinger'] == -1) & (data['fuzz_BB_strategy'] == -1) | (data['dec_bollinger'] == 1) & (data['fuzz_BB_strategy'] == 1)).sum()

print("bb_count", bb_count)


rsi_count 1168
williams_count 2894
ma_count 43
macd_count 11
bb_count 442


In [105]:
# 1 (zgodne znaki) 0 (różne znaki)
data['RSI_same_sign'] = (data['dec_rsi'] == data['Signal_RSI_fuzz_system']).astype(int)
data['Williams_same_sign'] = (data['dec_will'] == data['Signal_william_fuzz_system']).astype(int)
data['MA_same_sign'] = (data['cross_MA'] == data['fuzz_MA_strategy']).astype(int)
data['MACD_same_sign'] = (data['MACD_decision'] == data['fuzz_MACD_strategy']).astype(int)
data['BB_same_sign'] = (data['dec_bollinger'] == data['fuzz_BB_strategy']).astype(int)

# Sprawdzanei czy wszystkie wskazniki maja ten sam znak
data['All_same_sign'] = (
    data['RSI_same_sign'] & 
    data['Williams_same_sign'] & 
    data['MA_same_sign'] & 
    data['MACD_same_sign'] & 
    data['BB_same_sign']
)

In [54]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = np.nan

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ((data['dec_rsi'] == -1) & (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['dec_will'] == -1) & (data['Signal_william_fuzz_system'] == -1) &
                     (data['cross_MA'] == -1) & (data['fuzz_MA_strategy'] == -1) &
                     (data['MACD_decision'] == -1) & (data['fuzz_MACD_strategy'] == -1) &
                     (data['dec_bollinger'] == -1) & (data['fuzz_BB_strategy'] == -1))

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_rsi'] == 1) & (data['Signal_RSI_fuzz_system'] == 1) &
                     (data['dec_will'] == 1) & (data['Signal_william_fuzz_system'] == 1) &
                     (data['cross_MA'] == 1) & (data['fuzz_MA_strategy'] == 1) &
                     (data['MACD_decision'] == 1) & (data['fuzz_MACD_strategy'] == 1) &
                     (data['dec_bollinger'] == 1) & (data['fuzz_BB_strategy'] == 1))

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 0
all_zeros = ((data['dec_rsi'] == 0) & (data['Signal_RSI_fuzz_system'] == 0) &
             (data['dec_will'] == 0) & (data['Signal_william_fuzz_system'] == 0) &
             (data['cross_MA'] == 0) & (data['fuzz_MA_strategy'] == 0) &
             (data['MACD_decision'] == 0) & (data['fuzz_MACD_strategy'] == 0) &
             (data['dec_bollinger'] == 0) & (data['fuzz_BB_strategy'] == 0))

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.loc[all_zeros, 'all_decisions'] = 0


In [88]:
all_positive_ones.value_counts()

False    1284
Name: count, dtype: int64

In [85]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_sharp'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ((data['dec_rsi'] == -1) &
                     (data['dec_will'] == -1) & 
                     (data['cross_MA'] == -1) & 
                     (data['MACD_decision'] == -1) &
                     (data['dec_bollinger'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_rsi'] == 1) & 
                     (data['dec_will'] == 1) & 
                     (data['cross_MA'] == 1) & 
                     (data['MACD_decision'] == 1) & 
                     (data['dec_bollinger'] == 1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 0
all_zeros = ((data['dec_rsi'] == 0) & 
             (data['dec_will'] == 0) & 
             (data['cross_MA'] == 0) & 
             (data['MACD_decision'] == 0) & 
             (data['dec_bollinger'] == 0) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_sharp'] = -1
data.loc[all_positive_ones, 'all_decisions_sharp'] = 1
data.loc[all_zeros, 'all_decisions_sharp'] = 0


In [86]:
data.all_decisions_sharp.value_counts()

all_decisions_sharp
0    1284
Name: count, dtype: int64

In [141]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_MA_strategy'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) &
                     (data['fuzz_BB_strategy'] == -1))

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_MA_strategy'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) &
                     (data['fuzz_BB_strategy'] == 1))

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 0
# all_zeros = ((data['dec_rsi'] == 0) & (data['Signal_RSI_fuzz_system'] == 0) &
#              (data['dec_will'] == 0) & (data['Signal_william_fuzz_system'] == 0) &
#              (data['cross_MA'] == 0) & (data['fuzz_MA_strategy'] == 0) &
#              (data['MACD_decision'] == 0) & (data['fuzz_MACD_strategy'] == 0) &
#              (data['dec_bollinger'] == 0) & (data['fuzz_BB_strategy'] == 0))

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
#data.loc[all_zeros, 'all_decisions_fuzz'] = 0


In [142]:
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

# Kombinacje 2 wskaznikow
## FUZZY

In [197]:

data = pd.read_csv('data/decisions_EURUSD_1H.csv')
#data = pd.read_csv('data/decisions_EURUSD_5Y.csv')


data = data.set_index(pd.DatetimeIndex(data['data_datetime']))
#data = data.set_index(pd.DatetimeIndex(data['Date']))

In [182]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['Signal_william_fuzz_system'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['Signal_william_fuzz_system'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
 0    4075
-1     686
 1     537
Name: count, dtype: int64

In [184]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['fuzz_MA_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['fuzz_MA_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
 0    5286
 1       7
-1       5
Name: count, dtype: int64

In [186]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [188]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
 0    5015
-1     199
 1      84
Name: count, dtype: int64

In [191]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_MA_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_MA_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
 0    5285
-1       9
 1       4
Name: count, dtype: int64

In [192]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [193]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
 0    4870
-1     311
 1     117
Name: count, dtype: int64

In [194]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['fuzz_MA_strategy'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['fuzz_MA_strategy'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
 0    5295
-1       2
 1       1
Name: count, dtype: int64

In [195]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['fuzz_MACD_strategy'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['fuzz_MACD_strategy'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [199]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe -1
all_negative_ones = ( (data['fuzz_MA_strategy'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie oba wskaźniki są równe 1
all_positive_ones = ((data['fuzz_MA_strategy'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

## 3 Wskaźniki - kombinacje
rozmyte

In [221]:

data = pd.read_csv('data/decisions_EURUSD_1H.csv')
#data = pd.read_csv('data/decisions_EURUSD_5Y.csv')


data = data.set_index(pd.DatetimeIndex(data['data_datetime']))
#data = data.set_index(pd.DatetimeIndex(data['Date']))

In [ ]:
RSI %R MA -1 3
RSI %R MACD 0 
RSI %R BB -1  197  1 82
RSI MA MACD 0 
RSI MA BB 0
RSI MACD BB 0 
%R MA MACD 0
%R MA BB 0
%R MACD BB 0
MA MACD BB 0

In [203]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_MA_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_MA_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
 0    5295
-1       3
Name: count, dtype: int64

In [208]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [210]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
 0    5019
-1     197
 1      82
Name: count, dtype: int64

In [212]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['fuzz_MA_strategy'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['fuzz_MA_strategy'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1

data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [213]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['fuzz_MA_strategy'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['fuzz_MA_strategy'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [215]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_RSI_fuzz_system'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_RSI_fuzz_system'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [217]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_MA_strategy'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_MA_strategy'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [218]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_MA_strategy'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_MA_strategy'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [219]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['Signal_william_fuzz_system'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['Signal_william_fuzz_system'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

In [222]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions_fuzz'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['fuzz_MA_strategy'] == -1) &
                     (data['fuzz_MACD_strategy'] == -1) &
                     (data['fuzz_BB_strategy'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['fuzz_MA_strategy'] == 1) &
                     (data['fuzz_MACD_strategy'] == 1) &
                     (data['fuzz_BB_strategy'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions_fuzz'
data.loc[all_negative_ones, 'all_decisions_fuzz'] = -1
data.loc[all_positive_ones, 'all_decisions_fuzz'] = 1
data.all_decisions_fuzz.value_counts()

all_decisions_fuzz
0    5298
Name: count, dtype: int64

# ostre

In [ ]:
RSI %R MA -1 3
RSI %R MACD 0 
RSI %R BB -1  197  1 82
RSI MA MACD 0 
RSI MA BB 0
RSI MACD BB 0 
%R MA MACD 0
%R MA BB 0
%R MACD BB 0
MA MACD BB 0

In [237]:

data = pd.read_csv('data/decisions_EURUSD_1H.csv')
#data = pd.read_csv('data/decisions_EURUSD_5Y.csv')


data = data.set_index(pd.DatetimeIndex(data['data_datetime']))
#data = data.set_index(pd.DatetimeIndex(data['Date']))

In [226]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_rsi'] == -1) &
                     (data['dec_will'] == -1) &
                     (data['cross_MA'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_rsi'] == 1) &
                     (data['dec_will'] == 1) &
                     (data['cross_MA'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
0    5298
Name: count, dtype: int64

In [227]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_rsi'] == -1) &
                     (data['dec_will'] == -1) &
                     (data['MACD_decision'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_rsi'] == 1) &
                     (data['dec_will'] == 1) &
                     (data['MACD_decision'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
 0    5286
-1       7
 1       5
Name: count, dtype: int64

In [233]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_rsi'] == -1) &
                     (data['dec_will'] == -1) &
                     (data['dec_bollinger'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_rsi'] == 1) &
                     (data['dec_will'] == 1) &
                     (data['dec_bollinger'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
 0    5029
-1     140
 1     129
Name: count, dtype: int64

In [229]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_rsi'] == -1) &
                     (data['cross_MA'] == -1) &
                     (data['MACD_decision'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_rsi'] == 1) &
                     (data['cross_MA'] == 1) &
                     (data['MACD_decision'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1

data.all_decisions.value_counts()

all_decisions
0    5298
Name: count, dtype: int64

In [230]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_rsi'] == -1) &
                     (data['cross_MA'] == -1) &
                     (data['dec_bollinger'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_rsi'] == 1) &
                     (data['cross_MA'] == 1) &
                     (data['dec_bollinger'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
0    5298
Name: count, dtype: int64

In [231]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_rsi'] == -1) &
                     (data['MACD_decision'] == -1) &
                     (data['dec_bollinger'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_rsi'] == 1) &
                     (data['MACD_decision'] == 1) &
                     (data['dec_bollinger'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
0    5298
Name: count, dtype: int64

In [232]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_will'] == -1) &
                     (data['cross_MA'] == -1) &
                     (data['MACD_decision'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_will'] == 1) &
                     (data['cross_MA'] == 1) &
                     (data['MACD_decision'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
0    5298
Name: count, dtype: int64

In [234]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_will'] == -1) &
                     (data['cross_MA'] == -1) &
                     (data['dec_bollinger'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_will'] == 1) &
                     (data['cross_MA'] == 1) &
                     (data['dec_bollinger'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
0    5298
Name: count, dtype: int64

In [235]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['dec_will'] == -1) &
                     (data['MACD_decision'] == -1) &
                     (data['dec_bollinger'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['dec_will'] == 1) &
                     (data['MACD_decision'] == 1) &
                     (data['dec_bollinger'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
0    5298
Name: count, dtype: int64

In [238]:
# Inicjalizacja nowej kolumny z domyślnymi wartościami
data['all_decisions'] = 0

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe -1
all_negative_ones = ( (data['cross_MA'] == -1) &
                     (data['MACD_decision'] == -1) &
                     (data['dec_bollinger'] == -1) )

# Sprawdzenie miejsc, gdzie wszystkie wskaźniki są równe 1
all_positive_ones = ((data['cross_MA'] == 1) &
                     (data['MACD_decision'] == 1) &
                     (data['dec_bollinger'] == 1) )

# Aktualizacja nowej kolumny 'all_decisions'
data.loc[all_negative_ones, 'all_decisions'] = -1
data.loc[all_positive_ones, 'all_decisions'] = 1
data.all_decisions.value_counts()

all_decisions
0    5298
Name: count, dtype: int64